In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import re
import time
import pickle
import csv

BATCH_SIZE = 16 # 16 in orig paper
EPOCHS = 3
#Tx = 450 #length of input sequence (article)
Ty = 50 #length of output sequence (abstract)
Tx = 400
Ty = 100 # +1 for <eos> token

max_global_oov = 100

VOCAB_SIZE = len(word_dict)


In [2]:
with open('dailymail_word_dict.pickle', 'rb') as f:
    word_dict = pickle.load(f)

index_dict = {}
for key, value in word_dict.items():
    index_dict[value] = key

In [4]:
articles, highlights  = tfds.as_numpy(tfds.load(
    'cnn_dailymail',
    split='validation',
    batch_size=-1,
    as_supervised=True,
))

In [15]:
len(articles)

287113

In [16]:
t0 = time.time()
#creating a dictionary from train
word_dict = {}
reg = r"[\w']+|[.,!?;]"

for i in range(len(articles)):
    for word in re.findall(reg, articles[i].decode('UTF-8')) + re.findall(reg, highlights[i].decode('UTF-8')):
        word = word.lower()

        if word not in word_dict.keys(): word_dict[word] = 0

        word_dict[word] += 1

vocab_size = 50000

word_dict = {k : v for k, v in sorted(word_dict.items(), key=lambda x: x[1], reverse=True)[:vocab_size]} #sorting keys by count of occurences 
for i, (word, _) in enumerate(word_dict.items()): word_dict[word] = i + 4 #replacing count values by unique token values
#special tokens
word_dict["<sos>"] = 1 # start of sequence !later used in model
word_dict["<eos>"] = 2 # end of sequence
word_dict["<unk>"] = 3 # unknown token/word
word_dict["<pad>"] = 0 # padding

with open('dailymail_word_dict.pickle', 'wb') as f:
    pickle.dump(word_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

time.time() - t0

180.8127772808075

In [18]:
for key, value in list(word_dict.items())[0:10]: #removing number > x from dict?, jména politiků,
    print(value, "\t", key)

4 	 .
5 	 the
6 	 ,
7 	 to
8 	 a
9 	 and
10 	 of
11 	 in
12 	 was
13 	 for


In [6]:
numpy2csv_dataset('dataset_dailymail/val.csv', word_dict, VOCAB_SIZE, Tx, Ty, max_global_oov)

In [5]:
def numpy2csv_dataset(dest_file, word_dict, vocab_size, Tx, Ty, max_global_oov):
    "writes dataset in form X, y, oov_cnt, oov_dict"
    
    def word_to_index(word):
        if word not in word_dict:
            return word_dict["<unk>"]
        else: return word_dict[word]

    #Ty -= 1 # one token reserved for <eos>

    with open(dest_file, 'w') as csv_file:
        #writer = csv.DictWriter(csv_file, fieldnames=columns, extrasaction='ignore')
        writer = csv.writer(csv_file)
        
        for article, highlight in zip(articles, highlights):
            oov_cnt = 0
            oov_vocab = {} #token index to word
            
            X_words = re.findall(r"[\w']+|[.,!?;]", article.decode('UTF-8').lower())[:Tx]
            y_words = re.findall(r"[\w']+|[.,!?;]", highlight.decode('UTF-8').lower())[:Ty]

            for i in range(Tx):
                if i < len(X_words):
                    index = word_to_index(X_words[i])
                    if index == 3:
                        if X_words[i] not in oov_vocab and oov_cnt < max_global_oov:
                            oov_cnt += 1
                            oov_vocab[X_words[i]] = vocab_size + oov_cnt #discarded -1
                        
                        if X_words[i] in oov_vocab:
                            X_words[i] = oov_vocab[X_words[i]]

                        else: X_words[i] = index
                    else:
                        X_words[i] = index
                else:
                    X_words.append(word_dict["<pad>"])

            eos_added = False
            for i in range(Ty):
                if i < len(y_words):
                    index = word_to_index(y_words[i])
                    if index == 3 and y_words[i] in oov_vocab:
                        y_words[i] = oov_vocab[y_words[i]]
                    else:
                        y_words[i] = index 
                elif not eos_added:
                    eos_added = True
                    y_words.append(word_dict["<eos>"])
                else:
                    y_words.append(word_dict["<pad>"])
            
            if not eos_added:
                y_words[-1] = word_dict["<eos>"]

            #y_words.append(word_dict["<eos>"])

            line = X_words + y_words + [oov_cnt]

            for key, value in oov_vocab.items():
                line.append(key)
                line.append(value)
            
            writer.writerow(line)